In [ ]:
import re
def clean_text(line):
    line = re.sub(r'-+',' ',line)
    #line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

In [ ]:

text = 'In contrast, further deletion of the C-terminal transactivation domain in the Pax5 mutants B8 and B9 can abolish transcriptional stimulation, whereas internal deletion of the conserved octapeptide motif (OP) or the partial homeodomain (HD) of Pax5 did not have any effect (Figure 3B).'
print(clean_text(text))

In [ ]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-v1.1")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
print("inputs", inputs)
print("len(inputs.input_ids[0])", len(inputs.input_ids[0]))
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)
print("labels", labels)
print("len(labels[0])", len(labels[0]))
outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

In [ ]:

import numpy as np
import random
def get_random_indices(sentence):
    # Get the length of the sentence
    sentence_length = len(sentence.split())
  
    masked_idx = random.sample(range(sentence_length), 10)
    print("masked_idx", masked_idx)
    # Shuffle the indices of the sentence
    indices = np.arange(0, sentence_length-1, 1)
    np.random.shuffle(indices)

    # Select the first 10 unique indices
    random_indices = indices[:10]

    # Ensure that the selected indices are distinct
    while len(np.unique(random_indices)) < 10:
        np.random.shuffle(indices)
        random_indices = indices[:10]

    return random_indices


sentences = [
    "This is a sample sentence This is a sample sentence This is a sample sentence.",
    "Another example sentence with more words This is a sample sentence This is a sample sentence.",
    "A third sentence to demonstrate the process This is a sample sentence This is a sample sentence.",
]

for sentence in sentences:
    random_indices = get_random_indices(sentence)
    print(f"Sentence: {sentence}")
    print(f"Random Indices: {random_indices}")


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

sentences = [["This is the first sentence."], ['This is the sencond sentence.']] 
encoded_inputs = tokenizer(sentences, padding="max_length", truncation=True)

print(encoded_inputs)


In [23]:
import torch
import torch.nn.functional as F

# Example shapes
input_ids = torch.randn(32, 85)  # Assuming 32 batches, each with a sequence length of 85
logits = outputs.logits[0]  # Example logits, shape [85, 28996]
labels = torch.randint(0, 28996, (32, 85))  # Assuming labels for each position in the sequence

# Ensure that logits match the number of classes in your task
num_classes = logits.size(1)

# Transpose logits to be [sequence_length, batch_size, num_classes]
logits = logits.transpose(0, 1)

# Flatten the logits and labels for the loss calculation
logits_flat = logits.contiguous().view(-1, num_classes)
labels_flat = labels.view(-1)

# Assuming a simple sequence classification task
# You can use CrossEntropyLoss along the sequence dimension
loss_fn = torch.nn.CrossEntropyLoss()

# Calculate the loss
loss = loss_fn(logits_flat, labels_flat)

# Your training/update step here


NameError: name 'outputs' is not defined

In [ ]:
import torch

# Example tensor with size [32, 85]
original_tensor = torch.randn(32, 85)
print(original_tensor)  
# Sum along the second dimension (axis 1)
summed_tensor = torch.sum(original_tensor, dim=1)
print(summed_tensor)
# Check the size of the resulting tensor
print(summed_tensor.size())  # Should print torch.Size([32])



In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# Load pre-trained model and tokenizer
model_name = 'dmis-lab/biobert-base-cased-v1.2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# Input text with a [MASK] token
text = "The [MASK] is a large mammal native to North America."

# Tokenize the input text
tokens = tokenizer(text, return_tensors='pt')

# Get the model's output logits
with torch.no_grad():
    outputs = model(**tokens)
    logits = outputs.logits

# Assuming you have a [MASK] token in the input, find its position
mask_position = tokens['input_ids'][0].tolist().index(tokenizer.mask_token_id)
print("mask_position", mask_position)
# Extract the logits for the masked position
masked_logits = logits[0, mask_position]

# Apply softmax to get probabilities
probabilities = torch.nn.functional.softmax(masked_logits, dim=-1)

# Get the token with the highest probability (predicted token)
predicted_token_id = torch.argmax(probabilities).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]

# Replace the [MASK] token with the predicted token in the original text
result_text = text.replace('[MASK]', predicted_token)

# Print results
print("Original Text:", text)
print("Result Text:", result_text)
print("Predicted Token:", predicted_token)
print("Probabilities:", probabilities)



In [9]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
mask = "The capital of France is [MASK]."
# inputs = tokenizer.encode_plus(mask, add_special_tokens=True,
#                                     truncation_strategy ='only_first',
#                                     max_length = 11, padding='max_length', return_tensors="pt") 
inputs = tokenizer(mask, return_tensors="pt", max_length=11, padding='max_length')
print(len(inputs.input_ids))

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("The capital of France is Paris.", return_tensors="pt", max_length=11)["input_ids"]
print("labels", labels)
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

# [ 101, 1996, 3007, 1997, 2605, 2003,  103, 1012,  102]

# [101, 1103, 2364, 1104, 175, 10555, 1110, 103, 119, 102] dmislab token của ngta


1
labels tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119,
           102]])


11.01

In [6]:
print(inputs)

{'input_ids': [101, 1056, 1044, 1041, 1039, 1037, 1052, 1045, 1056, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [54]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")

inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt", max_length=11, padding='max_length')
print(inputs)
with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)
print(tokenizer.decode(predicted_token_id))
print("predicted token id: ", predicted_token_id)
labels = tokenizer("The capital of France is Paris.", return_tensors="pt", max_length=11, padding='max_length')["input_ids"]

print("labels: ", labels)
# mask labels of non-[MASK] tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)
round(outputs.loss.item(), 2)

#tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110,   103,   119,   102, 0]]
#tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119, 102]])

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'input_ids': tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110,   103,   119,   102,
             0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}
unknown
predicted token id:  tensor([3655])
labels:  tensor([[  101,  1103,  2364,  1104,   175, 10555,  1110, 14247,  1548,   119,
           102]])


11.01

## import library

In [1]:
from argparse import ArgumentParser
import math
from pathlib import Path
import time
import torch
import logging
import json
import random
import numpy as np
import pandas as pd
from collections import namedtuple, defaultdict
from tempfile import TemporaryDirectory
from sklearn.metrics import accuracy_score
from babel.dates import format_time
import torch.nn as nn
import torch
import sys
from scipy.special import softmax
import matplotlib.pyplot as plt
sys.path.insert(1, '../')

# sys.path.insert(1, '/content/SRLPredictionEasel')

# from bert_mlm_finetune import BertForMLMPreTraining 
from transformers import BertTokenizer, BertConfig, AdamW, get_linear_schedule_with_warmup, BertForMaskedLM 
from utils_mlm import count_num_cpu_gpu
import spacy
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter()

nlp = spacy.load("en_core_web_sm")
tokenizer = BertTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2", do_lower_case=True)

MLM_IGNORE_LABEL_IDX = -1
VOCAB_SIZE = 28996 
BATCH_SIZE = 32
EPOCHS = 2
MAX_SEQ_LEN = 85
NUM_CPU = count_num_cpu_gpu()[0]

/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-14 13:08:01.125343: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 13:08:03.440760: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 13:08:06.716099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

## define get pos tag func

## is POS match 

In [2]:


def is_POS_match(logits, input_ids, lm_label_ids):
    '''
    Function to check if the POS tag of the masked token in the logits is the same as the POS tag of the masked token in the original text.
    Note: This function assumes that the logits are of shape # ([85, 28996]) 
    lm_label_ids: shape (batch_size, sequence_length)
    '''
    
    origin_input_id = input_ids.clone() # Origin input id:  torch.Size([85])
   
    # Find the index of the masked token from lm_label_ids
    masked_idx = torch.where(lm_label_ids != -100)[0]
    masked_idx_input = torch.where(input_ids == tokenizer.mask_token_id)[0]
   
    origin_input_id[masked_idx_input] = lm_label_ids[masked_idx] 
    
    # get pos tag of origin text
    text = tokenizer.decode(input_ids)
    origin_text = tokenizer.decode(origin_input_id) 
    print("ORIGIN TEXT: ", origin_text)
    pos_tag_origin = get_pos_tag(origin_text, masked_idx_input)
    print("POS TAG ORIGIN: ", pos_tag_origin)
   
    # Extract the logits for the masked position
    masked_logits = logits[0, masked_idx]
    print("MASKED LOGITS: ", masked_logits) # torch.Size([28996])
    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(masked_logits, dim=-1)

    # Get the token with the highest probability (predicted token)
    predicted_token_id = torch.argmax(probabilities).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_token_id])[0]

    # Replace the [MASK] token with the predicted token in the original text
    result_text = text.replace('[MASK]', predicted_token) 
    print("RESULT TEXT: ", result_text)
    
    # get pos tag of logits
    logits_tag = get_pos_tag(result_text, masked_idx)
    print("LOGITS TAGS: ", logits_tag)
    return pos_tag_origin == logits_tag    


## custome loss

In [5]:
import torch.nn.functional as F
def custom_loss(input_ids, logits, labels):
  
    # Cross-entropy term
    cross_entropy_term = F.cross_entropy(logits, labels, reduction='none')
    print("Cross entropy term shape: ", cross_entropy_term.shape)      ##Cross entropy term shape:  torch.Size([2720])
    # logits_shape = (32, 85, VOCAB_SIZE)
    # logits_tensor = logits.view(*logits_shape)
    
    # labels_shape = (32, 85)
    # labels_tensor = labels.view(*labels_shape)
    
    
    # Custom matching term
    matching_term_lst = []
    # for logit, input_id, label in zip(logits_tensor, input_ids, labels_tensor):
       
    matching_term = is_POS_match(logits=logits, input_ids=input_ids, lm_label_ids=labels)
    print("Matching term: ", matching_term) 
    matching_term_lst.append(matching_term) 
    # hay mình thử sửa cái POS cho cái batch luôn kh, tại cái logit với cái label truyền vô là cái batch á
    # mà t sợ nhiều khi mình reshape sai nên nó tính sai 
    matching_term = torch.tensor(matching_term_lst)
    # Combine terms
    loss = 0.5 * cross_entropy_term + (1 - matching_term)
    return loss

## train with costom loss

In [37]:

def get_tokens_for_words(words, input_ids, offsets):
    '''
    Input:
        words = ['The', 'capital', 'of', 'the', 'France', 'is', 'Paris', '.']
        input_ids = [101, 1109, 3007, 1104, 2605, 1110, 3000, 119, 102]
        offsets = [(None, None), (0, 3), (4, 11), (12, 14), (15, 17), (18, 23), (24, 25), (None, None)]
    Output: 
        word_dict = {'The': [tensor(101), tensor(170), tensor(170)], 'capital': [tensor(1109), tensor(3007)], ....}
    '''
    word_dict = {}
    current_word = ''
    token_list = []
    sentence = ' '.join(words)
    print("input id shape: ", input_ids.shape)
    for j, (token, offset) in enumerate(zip(input_ids, offsets)):
        
        if offset[0] is not None:  # If the token is associated with a word in the original text
            start, end = offset
            original_word = sentence[start:end]
            # original_word = tokenizer.decode(input_ids[j], skip_special_tokens=True)
            if j > 0 and offset[0] == offsets[j - 1][1]:  # Check if the current word should be concatenated
                current_word += original_word
                token_list.append(token)
            else:
                if current_word:
                    if current_word in word_dict:
                        word_dict[current_word].append(token_list)
                    else:
                        word_dict[current_word] = [token_list]
                current_word = original_word
                token_list = [token]

    # Add the last word
    if current_word:
        if current_word in word_dict:
            word_dict[current_word].append(token_list)
        else:
            word_dict[current_word] = [token_list]

    return {word: [item for sublist in tokens for item in sublist]  for word, tokens in word_dict.items() }


In [2]:
from transformers import BertForMaskedLM, BertTokenizerFast
import torch
import copy
model = BertForMaskedLM.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
tokenizer = BertTokenizerFast.from_pretrained('dmis-lab/biobert-base-cased-v1.2')


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-25 11:52:42.560449: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 11:52:43.040984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 11:52:44.807674: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

In [ ]:
import torch
from torch.utils.data import Dataset
import json
import numpy as np
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = []
        with open(csv_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                data_dict = json.loads(line)
                data_dict['token_id'] = json.loads(data_dict['token_id'])
                data_dict['attention_mask'] = json.loads(data_dict['attention_mask'])
                data_dict['token_type_ids'] = json.loads(data_dict['token_type_ids'])
                data_dict['labels'] = json.loads(data_dict['labels'])
                self.data.append(data_dict)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx]
        token_id = torch.tensor(sample['token_id'], dtype=torch.long)
        attention_mask = torch.tensor(sample['attention_mask'], dtype=torch.long)
        token_type_ids = torch.tensor(sample['token_type_ids'], dtype=torch.long)
        labels = torch.tensor(sample['labels'], dtype=torch.long)
        return token_id, attention_mask, token_type_ids, labels

# Example usage:
csv_file = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/mlm_output_2/mlm_abolish_full.csv'
dataset = CustomDataset(csv_file)

# Now you can use PyTorch DataLoader to iterate over the dataset
# For example:
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
# for token_id, attention_mask, token_type_ids, labels in dataloader:
#     # Your training/validation loop here


In [42]:
import csv
import pandas as pd
def read_csv_file(file_path):
    """
    Reads a CSV file with the specified format.
    
    Args:
    - file_path (str): The path to the CSV file.
    
    Returns:
    - data (list of tuples): A list containing tuples of token_id, attention_mask, token_type_ids, and labels.
    """
    data = []
    
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            token_id = list(map(int, row['token_id'][1:-1].split(', ')))
            attention_mask = list(map(int, row['attention_mask'][1:-1].split(', ')))
            token_type_ids = list(map(int, row['token_type_ids'][1:-1].split(', ')))
            labels = list(map(int, row['labels'][1:-1].split(', ')))
            
            data.append((token_id, attention_mask, token_type_ids, labels))
    
    # Convert list of tuples to DataFrame
    df = pd.DataFrame(data, columns=['token_id', 'attention_mask', 'token_type_ids', 'labels'])
    return df

# Example usage:
file_path = '/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/mlm_output_2/mlm_abolish_full.csv'

data = read_csv_file(file_path)
print(data['token_id'])


0       [101, 170, 103, 118, 1106, 118, 170, 6468, 112...
1       [101, 170, 176, 118, 1106, 118, 170, 103, 1120...
2       [101, 170, 176, 118, 1106, 118, 170, 6468, 112...
3       [101, 170, 176, 118, 1106, 118, 170, 6468, 112...
4       [101, 170, 176, 118, 1106, 118, 170, 6468, 112...
                              ...                        
1059    [101, 1142, 27553, 7770, 17895, 117, 1134, 145...
1060    [101, 1142, 27553, 7770, 17895, 117, 1134, 145...
1061    [101, 1142, 27553, 7770, 17895, 117, 1134, 145...
1062    [101, 1142, 27553, 7770, 17895, 117, 1134, 145...
1063    [101, 1142, 27553, 7770, 17895, 117, 1134, 145...
Name: token_id, Length: 1064, dtype: object


In [1]:
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler


# Prepare validation dataloader
train_dataset_thu = CustomDataset(csv_file='/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/mlm_prepared_data_2/train_mlm.json')

print(len(train_dataset_thu))


NameError: name 'CustomDataset' is not defined

In [38]:
import copy

from prepared_for_mlm import get_pos_tag, mask_content_words

def masking_sentence_word(words, input_ids, offsets, tokenizer):
    '''
    Input: 
        words = ['The', 'capital', 'of', 'France', 'is', 'Paris', '.']
    Output: 
        masked_sentences = [tensor([  101,  1103, 175, 10555,  1110,   103,   119,   102]), [  101,  1103,  2364, 10555,  103,   103,   119,   102]] 
        label_ids = [tensor(6468)], [tensor(1568), tensor(13892)]
    '''
    # get a list of token for the word
    word_dict = get_tokens_for_words(words, input_ids, offsets)
   
    # masked the tokens if the word is the content word
    masked_sentences, label_ids, masked_indices = mask_content_words(input_ids, word_dict, tokenizer)
    
    return masked_sentences, label_ids


In [16]:
token_ids = [101, 1103, 173, 1605, 1104, 103, 22940, 1174, 20942, 113, 103, 103, 103, 114, 1336, 1138, 1151, 1215, 1106, 2423, 11303, 174, 119, 1884, 2646, 119, 102]

sample_token_id = [str(x) for x in token_ids]

print(sample_token_id)


['101', '1103', '173', '1605', '1104', '103', '22940', '1174', '20942', '113', '103', '103', '103', '114', '1336', '1138', '1151', '1215', '1106', '2423', '11303', '174', '119', '1884', '2646', '119', '102']


In [5]:
# encode xong mới mask
### THEO CAI LOSS NAY NHA PHAT OIIIIIII
# https://discuss.huggingface.co/t/bertformaskedlm-s-loss-and-scores-how-the-loss-is-computed/607/2
import spacy

from transformers import AutoTokenizer, BertForMaskedLM
import torch

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
model = BertForMaskedLM.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
nlp = spacy.load("en_core_web_sm")


def get_tokens_for_words(words, input_ids, offsets, tokenizer):
    '''
    Input:
        words = ['The', 'capital', 'of', 'the', 'France', 'is', 'Paris', '.']
        input_ids = [101, 1109, 3007, 1104, 2605, 1110, 3000, 119, 102]
        offsets = [(None, None), (0, 3), (4, 11), (12, 14), (15, 17), (18, 23), (24, 25), (None, None)]
    Output: 
        word_dict = {'The': [tensor(101), tensor(170), tensor(170)], 'capital': [tensor(1109), tensor(3007)], ....}
    '''
    

    word_dict = {}
    current_word = ''
    token_list = []
    sentence = ' '.join(words)
    print("sentence: ", sentence)
    except_tokens = [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id, tokenizer.unk_token_id]
    for j, (token, offset) in enumerate(zip(input_ids, offsets)):
       
        if offset[0] is not None:  # If the token is associated with a word in the original text
            
            start, end = offset
            original_word = sentence[start:end]
            # original_word = tokenizer.decode(input_ids[j], skip_special_tokens=True)
            if j > 0 and offset[0] == offsets[j - 1][1]:  # Check if the current word should be concatenated
                current_word += original_word
                token_list.append(token)
            else:
                if current_word:
                    word_dict[current_word] = [token_list]
                current_word = original_word
                token_list = [token]

    # Add the last word
    if current_word:
        word_dict[current_word] = [token_list]

    return {word: [item for sublist in tokens 
                            for item in sublist 
                                if item not in except_tokens ]  for word_idx, (word, tokens) in enumerate(word_dict.items()) }



def masking_sentence_word(words, input_ids, offsets, tokenizer):
    '''
    Input: 
        words = ['The', 'capital', 'of', 'France', 'is', 'Paris', '.']
    Output: 
        masked_sentences = [tensor([  101,  1103, 175, 10555,  1110,   103,   119,   102]), [  101,  1103,  2364, 10555,  103,   103,   119,   102]] 
        label_ids = [tensor(6468)], [tensor(1568), tensor(13892)]
    '''
    # get a list of token for the word
    word_dict = get_tokens_for_words(words, input_ids, offsets, tokenizer)
    print("WORD_DICT: ", word_dict)
    # masked the tokens if the word is the content word
    masked_sentences, label_ids, masked_indices = mask_content_words(input_ids, word_dict, tokenizer)
    
    return masked_sentences, label_ids, masked_indices, word_dict
## HÀM MAIN CỦA masking_sentence_word

# Example usage:
# sentence = "Cytokines altered the expression and activity of the multidrug resistance transporters in human hepatoma cell lines; analysis using RT-PCR and cDNA microarrays."
sentence = "A G-to-A transition at the first nucleotide of intron 2 of patient 1 abolished normal splicing."
doc = nlp(sentence)

# Get the POS tag for each word in the text
#words_str = [word.text for word in [token for token in doc]]
words_str = ['linomide', 'and', 'antibody', '-', 'targeted', 'superantigen', 'therapy', 'can', 'abolish', 'formation', 'of', 'liver', 'metastases', 'in', 'mice', '.']
tokenized_sentence = tokenizer.encode_plus(
    ' '.join(words_str),
    max_length=64,
    padding='max_length',  # Pad to the maximum sequence length
    truncation=True,  # Truncate to the maximum sequence length if necessary
    add_special_tokens = True,
    return_tensors="pt",  # Return PyTorch tensors
    return_attention_mask = True,
    return_offsets_mapping=True  # Return offset mappings
)
input_ids = tokenized_sentence['input_ids'][0]  

offsets = tokenized_sentence['offset_mapping'][0]  # Character offsets in the original input text

masked_sens, label_ids = masking_sentence_word(words_str, input_ids, offsets, tokenizer)

# test mask 0 với mask 3 

# labels  = copy.deepcopy(input_ids) #this is the part I changed
# input_ids[0][23] = tokenizer.mask_token_id
# labels[input_ids != tokenizer.mask_token_id] = -100 

# print("input_ids", input_ids)
# print("labels", labels)
# print("attention mask", input.attention_mask)
print("masked_sens", masked_sens[0])
print("label_ids", label_ids[0])

# convert 1-d tensor to 2-d tensor
masked_sens_2d = torch.unsqueeze(masked_sens[0], 0)
label_ids_2d = torch.unsqueeze(label_ids[0], 0)

output = model(input_ids = masked_sens[0], attention_mask = tokenized_sentence['attention_mask'], token_type_ids=tokenized_sentence['token_type_ids'], labels=label_ids[0])

for i in range(29):
    pred = torch.argmax(output.logits[0][i]).item()
    print(i, pred, tokenizer.decode(pred))
    
# cách 1
# logSoftmax = torch.nn.LogSoftmax(dim=1)
# NLLLos = torch.nn.NLLLoss()
# print(NLLLos( logSoftmax(torch.unsqueeze(output.logits[0][7], 0)), torch.tensor([pred]))) #the same as F.cross_entropy(scores.view(-1, tokenizer.vocab_size), labels.view(-1))

# cách 2: calculate loss manually
import torch.nn.functional as F
loss2 = F.cross_entropy(output.logits.view(-1, tokenizer.vocab_size), label_ids_2d.view(-1))
print(loss2)



Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


sentence:  linomide and antibody - targeted superantigen therapy can abolish formation of liver metastases in mice .
WORD_DICT:  {'linomide': [tensor(181), tensor(4559), tensor(3080), tensor(2007)], 'and': [tensor(1105)], 'antibody': [tensor(2848), tensor(14637)], '-': [tensor(118)], 'targeted': [tensor(9271)], 'superantigen': [tensor(7688), tensor(25711), tensor(4915)], 'therapy': [tensor(7606)], 'can': [tensor(1169)], 'abolish': [tensor(170), tensor(15792), tensor(2944)], 'formation': [tensor(3855)], 'of': [tensor(1104)], 'liver': [tensor(11911)], 'metastases': [tensor(27154), tensor(8419), tensor(8830)], 'in': [tensor(1107)], 'mice': [tensor(14105)], '.': [tensor(119)]}


TypeError: iteration over a 0-d tensor

In [14]:
print(torch.tensor(101) == torch.tensor(tokenizer.cls_token_id))

tensor(True)


In [11]:
print(torch.tensor(101))

tensor(101)


In [316]:
for i in range(len(masked_sens)):
    print(i, masked_sens[i], label_ids[i])

0 tensor([  101,   170,   103,   118,  1106,   118,   170,  6468,  1120,  1103,
         1148,   183, 21977, 26918, 23767,  1104, 27553,  1179,   123,  1104,
         5351,   122,  8632,  2999,   188,  1643, 22548,   119,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]) tensor([-100, -100,  176, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100])
1 tensor([  101,   170,   176,   118,  1106,   118,   170,  

In [2]:
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir "/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/logs_tb"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1203), started 0:00:21 ago. (Use '!kill 1203' to kill it.)

## MASK XONG MOI ENCODE

In [116]:
# mask xong mới encode

from transformers import BertForMaskedLM, BertTokenizerFast
import torch
model = BertForMaskedLM.from_pretrained('dmis-lab/biobert-base-cased-v1.2')
tokenizer = BertTokenizerFast.from_pretrained('dmis-lab/biobert-base-cased-v1.2')

# text = "A G-to-A transition at the first nucleotide of intron 2 of patient 1 abolished normal splicing.".lower()
text = "Cytokines altered the expression and activity of the multidrug resistance transporters in human hepatoma cell lines; analysis using RT-PCR and cDNA microarrays."
# Get the POS tag for each word in the text
doc = nlp(text)

# Get the POS tag for each word in the text
pos_tags = [token.pos_ for token in doc]

words = [token for token in doc]
mask_sens, labels_sens_, test_label = masking_sentence_word(words, tokenizer, pos_tags)  # mask là masked_sentence
        
print("mask_sens", mask_sens[-1])
# print("labels_sens", labels_sens[0])
           
input = tokenizer.encode_plus(text = ' '.join(mask_sens[-1]), return_tensors="pt", add_special_tokens = True, truncation=True, padding='max_length',
                                         return_attention_mask = True,  max_length=64) 
labels_sens  = copy.deepcopy(input['input_ids']) #this is the part I changed
labels_sens[input['input_ids'] == tokenizer.mask_token_id] = test_label
labels_sens[input['input_ids'] != tokenizer.mask_token_id] = -100 

print("input id: " ,input.input_ids)
print("labels sens: ", labels_sens)
print("attention mask", input.attention_mask)

# print len attention mask when it is not padding

output = model(input_ids = input['input_ids'], attention_mask = input['attention_mask'] , token_type_ids=input['token_type_ids'] , labels=labels_sens.clone().detach())

for i in range(29):
    pred = torch.argmax(output.logits[0][i]).item()
    print(i, tokenizer.decode(pred), pred)

print("output logits: ", output.logits.view(-1, tokenizer.vocab_size).shape)
print("labels sens: ", labels_sens.view(-1).shape)
# # loss 
import torch.nn.functional as F
loss2 = F.cross_entropy(output.logits.view(-1, tokenizer.vocab_size), labels_sens.view(-1))
print(loss2)
print()

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.2 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tmp_sen[i] altered
tmp_sen[i] expression
tmp_sen[i] activity
tmp_sen[i] resistance
tmp_sen[i] cell
tmp_sen[i] lines
tmp_sen[i] analysis
tmp_sen[i] using
mask_indexs 18
mask_sens ['Cytokines', 'altered', 'the', 'expression', 'and', 'activity', 'of', 'the', 'multidrug', 'resistance', 'transporters', 'in', 'human', 'hepatoma', 'cell', 'lines', ';', 'analysis', '[MASK]', 'RT', '-', 'PCR', 'and', 'cDNA', 'microarrays', '.']
input id:  tensor([[  101,   172, 25669, 21420,  3965,  8599,  1103,  2838,  1105,  3246,
          1104,  1103,  4321, 23632,  9610,  4789,  3936,  1468,  1107,  1769,
          1119,  4163, 18778,  1161,  2765,  2442,   132,  3622,   103,   187,
          1204,   118,   185,  1665,  1197,  1105,   172, 22834,  1161, 17599,
         25203,  6834,   119,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]])
labels sens:  tensor([[-100, -100, -100, -100, -100, 

In [20]:
print(output.logits.shape)

torch.Size([1, 64, 28996])


## test loss cross entropy
